# **Set Up**

Clone YOLOv5, install requirements, do imports

In [ ]:
# clone YOLOv5 repository

!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99

In [ ]:
%pip install -qr requirements.txt

import torch
import torchvision
import sys

import pandas as pd
import matplotlib.pyplot as plt

YOLOv5 uses an older version of numpy, so a file needs to be updated in order to work

In [ ]:
with open('./utils/dataloaders.py', 'r') as f:
    lines = f.readlines()

# change all instances of "np.int" with "np.int64"
for i, line in enumerate(lines):
    if 'np.int' in line:
        lines[i] = line.replace('np.int', 'np.int64')

# Write the changes back to the file
with open('./utils/dataloaders.py', 'w') as f:
    f.writelines(lines)


print("File updated")

System checks

In [ ]:
print(torch.__version__)
print(torchvision.__version__)
print(torch.cuda.is_available())  # true if CUDA is available
print(torch.cuda.get_device_name(0))  # name of GPU

print(sys.version) # currently python 3.11 doesn't work with CUDA

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

In [ ]:
## optional: ensures model is training on CUDA

from models.common import DetectMultiBackend

model = DetectMultiBackend('yolov5s.pt', device=device)
model.to(device)

# to confirm
print(model)

# **Training**

Train model with hyperparameters
- img size 416 (for speed)
- batch size 64
- 50 epochs
- yaml files that tell the model what data and model to use
- empty weights

In [ ]:
# start training
# you have to be in the ./yolov5 folder

!python train.py --img 416 --batch 64 --epochs 50 --data data/VOC.yaml --cfg models/yolov5s.yaml --weights ''

# **Plotting and Validation**

For analysing purposes, plots the mAP over the epochs

In [ ]:
# path to results.csv file (adjust path if necessary)
results_file = 'runs/train/exp26/results.csv'

df = pd.read_csv(results_file)

# strip leading/trailing spaces
df.columns = df.columns.str.strip()

plt.figure(figsize=(10, 6))

# plot mAP values
plt.subplot(2, 1, 2)
plt.plot(df['epoch'], df['metrics/mAP_0.5'], label='mAP@0.5', color='blue')
plt.plot(df['epoch'], df['metrics/mAP_0.5:0.95'], label='mAP@0.5:0.95', color='green')
plt.xlabel('Epochs')
plt.ylabel('mAP')
plt.title('mAP vs Epochs')
plt.legend()
plt.grid(True)

# show/save the plot
#plt.tight_layout()
#plt.show()
plt.savefig("mAP_vs_epochs.png")

To assess the model on a new set, validate it and collect resulting metrics

In [ ]:
# start validation
# you have to be in the ./yolov5 folder
# adjust weights path as necessary

!python val.py --img 416 --batch 64 --data data/VOC.yaml --weights ./runs/train/exp26/weights/best.pt